In [1]:
import subprocess
import yaml
from time import sleep
import os
import numpy as np

In [2]:
def ticcmd(*args): #Basic function to retrive current step position velocity etc etc
    return subprocess.check_output(['ticcmd'] + list(args))

def get_position(): #Function to get the current step position and return its value
    status = yaml.load(ticcmd('-s', '--full'))
    position = status['Current position']
    return position

def set_position(target_position): #Tells the stepper motor to move to a particular step position and prevents code execution until it has reached that position
    position = get_position()
    ticcmd('--resume', '--position', str(target_position))
    while position != target_position:
        position = get_position()
        print(position, end="\r")
        sleep(0.1)
    #sleep(0.5)
    #print(position)
    
def set_cavity_frequency(offset, target_frequency, co_3, co_2, co_1, co_0): #Function to set the cavity frequency using the parameters of a fitting function to convert frequency to step position
    frequency_y2_adjusted = int(np.rint(co_3*target_frequency**3 + co_2*target_frequency**2 + co_1*target_frequency + co_0 - offset))
    set_position(frequency_y2_adjusted)
    
def reset_position(initial_position): #Resets the position to the initial position, by going below the initial position then back up, to account for any give in the system
    set_position(initial_position - 200)
    set_position(initial_position)
    
def set_to_zero(): #Sets the current step position to 0
    zero_position = 0
    ticcmd('--exit-safe-start', '--halt-and-set-position', str(zero_position))

In [5]:
initial_frequency = 2.820676
initial_position = 0
frequencies = [initial_frequency, 2.821     , 2.82166443, 2.82232886, 2.82299329, 2.82365772,
       2.82432215, 2.82498658, 2.82565101, 2.82631544, 2.82697987,
       2.8276443 , 2.82830872, 2.82897315, 2.82963758, 2.83030201,
       2.83096644, 2.83163087, 2.8322953 , 2.83295973, 2.83362416,
       2.83428859, 2.83495302, 2.83561745, 2.83628188, 2.83694631,
       2.83761074, 2.83827517, 2.8389396 , 2.83960403, 2.84026846,
       2.84093289, 2.84159732, 2.84226174, 2.84292617, 2.8435906 ,
       2.84425503, 2.84491946, 2.84558389, 2.84624832, 2.84691275,
       2.84757718, 2.84824161, 2.84890604, 2.84957047, 2.8502349 ,
       2.85089933, 2.85156376, 2.85222819, 2.85289262, 2.85355705,
       2.85422148, 2.85488591, 2.85555034, 2.85621477, 2.85687919,
       2.85754362, 2.85820805, 2.85887248, 2.85953691, 2.86020134,
       2.86086577, 2.8615302 , 2.86219463, 2.86285906, 2.86352349,
       2.86418792, 2.86485235, 2.86551678, 2.86618121, 2.86684564,
       2.86751007, 2.8681745 , 2.86883893, 2.86950336, 2.87016779,
       2.87083221, 2.87149664, 2.87216107, 2.8728255 , 2.87348993,
       2.87415436, 2.87481879, 2.87548322, 2.87614765, 2.87681208,
       2.87747651, 2.87814094, 2.87880537, 2.8794698 , 2.88013423,
       2.88079866, 2.88146309, 2.88212752, 2.88279195, 2.88345638,
       2.88412081, 2.88478523, 2.88544966, 2.88611409, 2.88677852,
       2.88744295, 2.88810738, 2.88877181, 2.88943624, 2.89010067,
       2.8907651 , 2.89142953, 2.89209396, 2.89275839, 2.89342282,
       2.89408725, 2.89475168, 2.89541611, 2.89608054, 2.89674497,
       2.8974094 , 2.89807383, 2.89873826, 2.89940268, 2.90006711,
       2.90073154, 2.90139597, 2.9020604 , 2.90272483, 2.90338926,
       2.90405369, 2.90471812, 2.90538255, 2.90604698, 2.90671141,
       2.90737584, 2.90804027, 2.9087047 , 2.90936913, 2.91003356,
       2.91069799, 2.91136242, 2.91202685, 2.91269128, 2.9133557 ,
       2.91402013, 2.91468456, 2.91534899, 2.91601342, 2.91667785,
       2.91734228, 2.91800671, 2.91867114, 2.91933557, 2.92]

co_3 = 69278.68
co_2 = -609102.93
co_1 = 1790504.24
co_0 = - 1762367.85

frequency_y1 = int(np.rint(co_3*initial_frequency**3 + co_2*initial_frequency**2 + co_1*initial_frequency + co_0))
offset = frequency_y1 - initial_position

counts = [0] * len(frequencies)
number_of_runs = 2
integration_time = 2
counter_frequency = 1/integration_time
sleep_time = 4.5

In [14]:
set_cavity_frequency(offset, 2.87, co_3, co_2, co_1, co_0)

In [6]:
reset_position(initial_position)

In [7]:
counterlogic.set_count_length(300)
counterlogic.set_count_frequency(counter_frequency)
counterlogic.startCount()
smiq.cw_on()

0

In [7]:
smiq.set_cw(2.88e9, -20)

(2880000000.0, -20.0, 'cw')

In [8]:
smiq.cw_on()

0

In [8]:
for i in range(number_of_runs):
    for i in range(len(frequencies)):
        if i == 0:
            target_frequency = frequencies[i]
            smiq.set_cw(frequency=target_frequency*1e9)
            smiq.cw_on()
            sleep(sleep_time)
            counts[i] = counts[i] + counterlogic.countdata[0,-1]

        if i != 0:
            target_frequency = frequencies[i]
            set_cavity_frequency(offset, target_frequency, co_3, co_2, co_1, co_0)
            smiq.set_cw(frequency=target_frequency*1e9)
            smiq.cw_on()
            sleep(sleep_time)
            counts[i] = counts[i] + counterlogic.countdata[0,-1]
            
    reset_position(initial_position)
            
print(counts)
print(frequencies)

[248760.0, 248829.0, 249211.0, 249019.5, 248829.0, 248956.0, 248655.5, 249102.5, 249206.0, 248707.5, 248381.0, 248952.5, 248958.5, 248728.0, 249391.5, 249358.5, 248951.5, 249155.5, 249269.5, 249491.5, 248878.5, 248710.5, 248793.5, 249499.0, 249424.0, 249828.0, 249095.0, 249254.5, 249460.5, 248838.5, 248925.5, 248507.0, 248918.0, 249192.5, 248308.5, 248903.5, 248163.0, 249366.5, 249290.5, 248403.5, 248999.5, 249097.5, 249639.5, 248200.5, 249280.0, 249542.0, 248757.0, 249185.0, 248817.5, 249018.0, 248178.0, 249042.0, 248990.5, 249534.0, 249321.5, 249568.5, 248404.5, 248972.5, 248568.0, 249715.0, 248736.0, 249123.5, 249270.0, 248607.0, 248924.5, 248066.5, 247708.0, 247178.0, 247163.5, 246997.5, 245255.0, 245011.5, 245913.0, 244933.5, 245093.0, 245112.5, 245335.5, 245022.0, 244378.0, 246038.0, 246916.0, 248192.0, 247002.0, 247446.0, 247150.0, 247898.0, 248694.0, 248412.0, 247784.5, 248251.0, 248858.5, 247874.0, 248174.5, 248299.0, 248206.5, 248347.0, 248338.0, 249094.0, 249266.5, 249600.0,

In [ ]:
#Sets current position to 0
zero_position = 0
ticcmd('--exit-safe-start', '--halt-and-set-position', str(zero_position))

#Get current count parameters
counterlogic.get_count_length()
counterlogic.get_count_frequency()

# Brute force way to control anything, pull gui levers.
counter._mw.start_counter_Action.trigger()

In [5]:
get_position()

0

In [5]:
set_position(-400)

In [31]:
count = 4

In [32]:
count

4

In [49]:
target = get_position() + 100
count = count + 1

set_position(target)
print("The number is", count, "       ")

The number is 21        


In [21]:
smiq.cw_on()

0

In [14]:
set_to_zero()

In [ ]:
reset_position(-4800)

In [ ]:
smiq.off()


In [8]:
np.linspace(2.821, 2.92, 150)

array([2.821     , 2.82166443, 2.82232886, 2.82299329, 2.82365772,
       2.82432215, 2.82498658, 2.82565101, 2.82631544, 2.82697987,
       2.8276443 , 2.82830872, 2.82897315, 2.82963758, 2.83030201,
       2.83096644, 2.83163087, 2.8322953 , 2.83295973, 2.83362416,
       2.83428859, 2.83495302, 2.83561745, 2.83628188, 2.83694631,
       2.83761074, 2.83827517, 2.8389396 , 2.83960403, 2.84026846,
       2.84093289, 2.84159732, 2.84226174, 2.84292617, 2.8435906 ,
       2.84425503, 2.84491946, 2.84558389, 2.84624832, 2.84691275,
       2.84757718, 2.84824161, 2.84890604, 2.84957047, 2.8502349 ,
       2.85089933, 2.85156376, 2.85222819, 2.85289262, 2.85355705,
       2.85422148, 2.85488591, 2.85555034, 2.85621477, 2.85687919,
       2.85754362, 2.85820805, 2.85887248, 2.85953691, 2.86020134,
       2.86086577, 2.8615302 , 2.86219463, 2.86285906, 2.86352349,
       2.86418792, 2.86485235, 2.86551678, 2.86618121, 2.86684564,
       2.86751007, 2.8681745 , 2.86883893, 2.86950336, 2.87016

In [9]:
counterlogic.countdata[0,-1]

922329.0